In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("diabetes_012_health_indicators_BRFSS2015.csv")

In [4]:
df.columns

Index(['Diabetes_012', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker',
       'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
       'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
       'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education',
       'Income'],
      dtype='object')

In [7]:
df["Diabetes_012"].value_counts()

Diabetes_012
0.0    213703
2.0     35346
1.0      4631
Name: count, dtype: int64

In [9]:
213703 / (213703 + 35346 + 4631)

0.8424116997792495

In [3]:
from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from collections import Counter

X = df.drop('Diabetes_012', axis=1)
y = df['Diabetes_012']

print(f"Original class distribution: {Counter(y)}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize SMOTE
smote = SMOTE(random_state=42)

# Apply SMOTE to the training data
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Display new class distribution after applying SMOTE
print(f"Resampled class distribution: {Counter(y_resampled)}")

Original class distribution: Counter({0.0: 213703, 2.0: 35346, 1.0: 4631})
Resampled class distribution: Counter({2.0: 149523, 0.0: 149523, 1.0: 149523})


In [4]:
X, y = X_resampled, y_resampled

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build the DNN model
model = Sequential([
    Dense(32, input_dim=X_train.shape[1], activation='relu'),
    Dense(64, activation='relu'),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')  # Output layer for multi-class classification
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

Epoch 1/10
11215/11215 [==============================] - 23s 2ms/step - loss: 0.7735 - accuracy: 0.6289 - val_loss: 0.7016 - val_accuracy: 0.6657
Epoch 2/10
11215/11215 [==============================] - 22s 2ms/step - loss: 0.6733 - accuracy: 0.6816 - val_loss: 0.6575 - val_accuracy: 0.6914
Epoch 3/10
11215/11215 [==============================] - 21s 2ms/step - loss: 0.6338 - accuracy: 0.7043 - val_loss: 0.6296 - val_accuracy: 0.7052
Epoch 4/10
11215/11215 [==============================] - 24s 2ms/step - loss: 0.6037 - accuracy: 0.7213 - val_loss: 0.5983 - val_accuracy: 0.7230
Epoch 5/10
11215/11215 [==============================] - 24s 2ms/step - loss: 0.5794 - accuracy: 0.7352 - val_loss: 0.5726 - val_accuracy: 0.7376
Epoch 6/10
11215/11215 [==============================] - 25s 2ms/step - loss: 0.5590 - accuracy: 0.7464 - val_loss: 0.5599 - val_accuracy: 0.7445
Epoch 7/10
11215/11215 [==============================] - 22s 2ms/step - loss: 0.5413 - accuracy: 0.7558 - val_loss: 0

In [5]:
import streamlit as st
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.utils import to_categorical

y = to_categorical(y, num_classes=3)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

def build_custom_model(input_shape):
    # Input layer
    inputs = Input(shape=input_shape)
    
    # Branch 1: Handling physical health-related features
    x1 = Dense(64, activation='relu')(inputs)
    x1 = Dense(16, activation='relu')(x1)
    
    # Branch 2: Handling lifestyle-related features
    x2 = Dense(64, activation='relu')(inputs)
    x2 = Dense(16, activation='relu')(x2)
    
    # Branch 3: Handling demographic features
    x3 = Dense(64, activation='relu')(inputs)
    x3 = Dense(16, activation='relu')(x3)
    
    # Combine branches
    combined = Concatenate()([x1, x2, x3])
    
    # Fully connected layers
    x = Dense(128, activation='relu')(combined)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(3, activation='softmax')(x)
    
    # Build model
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

model = build_custom_model((X_train.shape[1], ))

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Model Accuracy: {accuracy:.4f}")




Epoch 1/10


11215/11215 [==============================] - 14s 1ms/step - loss: 0.7297 - accuracy: 0.6548 - val_loss: 0.6540 - val_accuracy: 0.6945
Epoch 2/10
11215/11215 [==============================] - 13s 1ms/step - loss: 0.6160 - accuracy: 0.7178 - val_loss: 0.5798 - val_accuracy: 0.7354
Epoch 3/10
11215/11215 [==============================] - 13s 1ms/step - loss: 0.5621 - accuracy: 0.7481 - val_loss: 0.5407 - val_accuracy: 0.7582
Epoch 4/10
11215/11215 [==============================] - 13s 1ms/step - loss: 0.5232 - accuracy: 0.7685 - val_loss: 0.5380 - val_accuracy: 0.7646
Epoch 5/10
11215/11215 [==============================] - 13s 1ms/step - loss: 0.4943 - accuracy: 0.7830 - val_loss: 0.4977 - val_accuracy: 0.7828
Epoch 6/10
11215/11215 [==============================] - 14s 1ms/step - loss: 0.4699 - accuracy: 0.7944 - val_loss: 0.4707 - val_accuracy: 0.7942
Epoch 7/10
11215/11215 [==============================] - 13s 1ms/step - loss: 0.4528 - accuracy: 0.8029 - val_lo

In [4]:
import streamlit as st
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.utils import to_categorical

y = to_categorical(y, num_classes=3)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

def build_custom_model(input_shape):
    # Input layer
    inputs = Input(shape=input_shape)
    
    # Branch 1: Handling physical health-related features
    x1 = Dense(128, activation='relu')(inputs)
    x1 = Dense(64, activation='relu')(x1)
    
    # Branch 2: Handling lifestyle-related features
    x2 = Dense(128, activation='relu')(inputs)
    x2 = Dense(64, activation='relu')(x2)
    
    # Branch 3: Handling demographic features
    x3 = Dense(128, activation='relu')(inputs)
    x3 = Dense(64, activation='relu')(x3)
    
    # Combine branches
    combined = Concatenate()([x1, x2, x3])
    
    # Fully connected layers
    x = Dense(128, activation='relu')(combined)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(3, activation='softmax')(x)
    
    # Build model
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

model = build_custom_model((X_train.shape[1], ))

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Model Accuracy: {accuracy:.4f}")




Epoch 1/10


6342/6342 [==============================] - 9s 1ms/step - loss: 0.4012 - accuracy: 0.8471 - val_loss: 0.3953 - val_accuracy: 0.8497
Epoch 2/10
6342/6342 [==============================] - 8s 1ms/step - loss: 0.3961 - accuracy: 0.8490 - val_loss: 0.3916 - val_accuracy: 0.8492
Epoch 3/10
6342/6342 [==============================] - 8s 1ms/step - loss: 0.3948 - accuracy: 0.8494 - val_loss: 0.3948 - val_accuracy: 0.8455
Epoch 4/10
6342/6342 [==============================] - 8s 1ms/step - loss: 0.3938 - accuracy: 0.8497 - val_loss: 0.3928 - val_accuracy: 0.8505
Epoch 5/10
6342/6342 [==============================] - 9s 1ms/step - loss: 0.3928 - accuracy: 0.8500 - val_loss: 0.3934 - val_accuracy: 0.8479
Epoch 6/10
6342/6342 [==============================] - 9s 1ms/step - loss: 0.3917 - accuracy: 0.8508 - val_loss: 0.3919 - val_accuracy: 0.8510
Epoch 7/10
6342/6342 [==============================] - 8s 1ms/step - loss: 0.3908 - accuracy: 0.8507 - val_loss: 0.3953 - val_accu

In [5]:
import streamlit as st
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.utils import to_categorical

y = to_categorical(y, num_classes=3)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

def build_custom_model(input_shape):
    # Input layer
    inputs = Input(shape=input_shape)
    
    # Branch 1: Handling physical health-related features
    x1 = Dense(128, activation='relu')(inputs)
    x1 = Dense(128, activation='relu')(x1)
    x1 = Dense(64, activation='relu')(x1)
    
    # Branch 2: Handling lifestyle-related features
    x2 = Dense(128, activation='relu')(inputs)
    x2 = Dense(128, activation='relu')(x2)
    x2 = Dense(64, activation='relu')(x2)
    
    # Branch 3: Handling demographic features
    x3 = Dense(128, activation='relu')(inputs)
    x3 = Dense(128, activation='relu')(x2)
    x3 = Dense(64, activation='relu')(x3)
    
    # Combine branches
    combined = Concatenate()([x1, x2, x3])
    
    # Fully connected layers
    x = Dense(128, activation='relu')(combined)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(3, activation='softmax')(x)
    
    # Build model
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

model = build_custom_model((X_train.shape[1], ))

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Model Accuracy: {accuracy:.4f}")




Epoch 1/10


11215/11215 [==============================] - 18s 1ms/step - loss: 0.7038 - accuracy: 0.6708 - val_loss: 0.6227 - val_accuracy: 0.7199
Epoch 2/10
11215/11215 [==============================] - 16s 1ms/step - loss: 0.5423 - accuracy: 0.7604 - val_loss: 0.4939 - val_accuracy: 0.7828
Epoch 3/10
11215/11215 [==============================] - 16s 1ms/step - loss: 0.4631 - accuracy: 0.7993 - val_loss: 0.4410 - val_accuracy: 0.8100
Epoch 4/10
11215/11215 [==============================] - 16s 1ms/step - loss: 0.4137 - accuracy: 0.8216 - val_loss: 0.4221 - val_accuracy: 0.8193
Epoch 5/10
11215/11215 [==============================] - 17s 2ms/step - loss: 0.3812 - accuracy: 0.8356 - val_loss: 0.3901 - val_accuracy: 0.8321
Epoch 6/10
11215/11215 [==============================] - 16s 1ms/step - loss: 0.3568 - accuracy: 0.8460 - val_loss: 0.3652 - val_accuracy: 0.8440
Epoch 7/10
11215/11215 [==============================] - 16s 1ms/step - loss: 0.3382 - accuracy: 0.8546 - val_lo

In [6]:
import streamlit as st
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.utils import to_categorical

def build_custom_model(input_shape):
    # Input layer
    inputs = Input(shape=input_shape)
    
    # Branch 1: Handling physical health-related features
    x1 = Dense(128, activation='relu')(inputs)
    x1 = Dense(128, activation='relu')(x1)
    x1 = Dense(64, activation='relu')(x1)
    
    # Branch 2: Handling lifestyle-related features
    x2 = Dense(128, activation='relu')(inputs)
    x2 = Dense(128, activation='relu')(x2)
    x2 = Dense(64, activation='relu')(x2)
    
    # Branch 3: Handling demographic features
    x3 = Dense(128, activation='relu')(inputs)
    x3 = Dense(128, activation='relu')(x2)
    x3 = Dense(64, activation='relu')(x3)
    
    # Combine branches
    combined = Concatenate()([x1, x2, x3])
    
    # Fully connected layers
    x = Dense(128, activation='relu')(combined)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(3, activation='softmax')(x)
    
    # Build model
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

model = build_custom_model((X_train.shape[1], ))

model.fit(X_train, y_train, epochs=16, batch_size=64, validation_data=(X_test, y_test))
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Model Accuracy: {accuracy:.4f}")

Epoch 1/40
5608/5608 [==============================] - 11s 2ms/step - loss: 0.7041 - accuracy: 0.6709 - val_loss: 0.6157 - val_accuracy: 0.7244
Epoch 2/40
5608/5608 [==============================] - 11s 2ms/step - loss: 0.5370 - accuracy: 0.7632 - val_loss: 0.5049 - val_accuracy: 0.7799
Epoch 3/40
5608/5608 [==============================] - 10s 2ms/step - loss: 0.4548 - accuracy: 0.8029 - val_loss: 0.4343 - val_accuracy: 0.8127
Epoch 4/40
5608/5608 [==============================] - 10s 2ms/step - loss: 0.4060 - accuracy: 0.8245 - val_loss: 0.4044 - val_accuracy: 0.8247
Epoch 5/40
5608/5608 [==============================] - 10s 2ms/step - loss: 0.3753 - accuracy: 0.8388 - val_loss: 0.3998 - val_accuracy: 0.8291
Epoch 6/40
5608/5608 [==============================] - 10s 2ms/step - loss: 0.3486 - accuracy: 0.8496 - val_loss: 0.3656 - val_accuracy: 0.8416
Epoch 7/40
5608/5608 [==============================] - 10s 2ms/step - loss: 0.3309 - accuracy: 0.8573 - val_loss: 0.3520 - val_ac

In [7]:
import streamlit as st
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.utils import to_categorical

def build_custom_model(input_shape):
    # Input layer
    inputs = Input(shape=input_shape)
    
    # Branch 1: Handling physical health-related features
    x1 = Dense(128, activation='relu')(inputs)
    x1 = Dense(128, activation='relu')(x1)
    x1 = Dense(64, activation='relu')(x1)
    
    # Branch 2: Handling lifestyle-related features
    x2 = Dense(128, activation='relu')(inputs)
    x2 = Dense(128, activation='relu')(x2)
    x2 = Dense(64, activation='relu')(x2)
    
    # Branch 3: Handling demographic features
    x3 = Dense(128, activation='relu')(inputs)
    x3 = Dense(128, activation='relu')(x3)
    x3 = Dense(64, activation='relu')(x3)
    
    x4 = Dense(128, activation='relu')(inputs)
    x4 = Dense(128, activation='relu')(x4)
    x4 = Dense(64, activation='relu')(x4)

    # Combine branches
    combined = Concatenate()([x1, x2, x3, x4])
    
    # Fully connected layers
    x = Dense(128, activation='relu')(combined)
    x = Dense(128, activation='relu')(x)
    outputs = Dense(3, activation='softmax')(x)
    
    # Build model
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

model = build_custom_model((X_train.shape[1], ))

model.fit(X_train, y_train, epochs=16, batch_size=64, validation_data=(X_test, y_test))
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Model Accuracy: {accuracy:.4f}")

Epoch 1/16
5608/5608 [==============================] - 12s 2ms/step - loss: 0.6804 - accuracy: 0.6871 - val_loss: 0.5425 - val_accuracy: 0.7614
Epoch 2/16
5608/5608 [==============================] - 11s 2ms/step - loss: 0.4847 - accuracy: 0.7892 - val_loss: 0.4608 - val_accuracy: 0.7960
Epoch 3/16
5608/5608 [==============================] - 11s 2ms/step - loss: 0.4005 - accuracy: 0.8276 - val_loss: 0.3785 - val_accuracy: 0.8367
Epoch 4/16
5608/5608 [==============================] - 12s 2ms/step - loss: 0.3539 - accuracy: 0.8479 - val_loss: 0.3662 - val_accuracy: 0.8429
Epoch 5/16
5608/5608 [==============================] - 11s 2ms/step - loss: 0.3245 - accuracy: 0.8606 - val_loss: 0.3281 - val_accuracy: 0.8587
Epoch 6/16
5608/5608 [==============================] - 11s 2ms/step - loss: 0.3030 - accuracy: 0.8695 - val_loss: 0.3156 - val_accuracy: 0.8641
Epoch 7/16
5608/5608 [==============================] - 12s 2ms/step - loss: 0.2850 - accuracy: 0.8772 - val_loss: 0.3153 - val_ac